In [18]:
# Load necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import stanza
from nltk.corpus import stopwords

In [19]:
stanza.download('sv', processors='tokenize,pos,lemma,depparse')
nlp = stanza.Pipeline(lang='sv', processors='tokenize,pos,lemma,depparse')

2022-02-07 13:37:14 INFO: Downloading these customized packages for language: sv (Swedish)...
| Processor | Package   |
-------------------------
| tokenize  | talbanken |
| pos       | talbanken |
| lemma     | talbanken |
| depparse  | talbanken |
| pretrain  | talbanken |

2022-02-07 13:37:14 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/tokenize/talbanken.pt.
2022-02-07 13:37:14 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/pos/talbanken.pt.
2022-02-07 13:37:14 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/lemma/talbanken.pt.
2022-02-07 13:37:14 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/depparse/talbanken.pt.
2022-02-07 13:37:15 INFO: File exists: /Users/filippakarrfelt/stanza_resources/sv/pretrain/talbanken.pt.
2022-02-07 13:37:15 INFO: Finished downloading models and saved to /Users/filippakarrfelt/stanza_resources.
2022-02-07 13:37:15 INFO: Loading these models for language: sv (Swedish):
| Processor | Package  

In [50]:
df_train = pd.read_json(r'../data/training.json', orient='split')
df_test = pd.read_json(r'../data/test.json', orient='split')
stop_words = set(stopwords.words('swedish'))
stop_words.add('vad')
stop_words.add('vems')
stop_words.add('varifrån')
stop_words.add('vemvilka') # one question has vem/vilka as question word..
# ['vad', 'hur', 'när', 'var', 'varifrån', 'varför', 'vart', 'vilken', 'vilket', 'vilka', 'vem', 'vems'] 
stopwords_list = list(stop_words)
stopwords_list.sort()
print(stopwords_list)

['alla', 'allt', 'att', 'av', 'blev', 'bli', 'blir', 'blivit', 'de', 'dem', 'den', 'denna', 'deras', 'dess', 'dessa', 'det', 'detta', 'dig', 'din', 'dina', 'ditt', 'du', 'där', 'då', 'efter', 'ej', 'eller', 'en', 'er', 'era', 'ert', 'ett', 'från', 'för', 'ha', 'hade', 'han', 'hans', 'har', 'henne', 'hennes', 'hon', 'honom', 'hur', 'här', 'i', 'icke', 'ingen', 'inom', 'inte', 'jag', 'ju', 'kan', 'kunde', 'man', 'med', 'mellan', 'men', 'mig', 'min', 'mina', 'mitt', 'mot', 'mycket', 'ni', 'nu', 'när', 'någon', 'något', 'några', 'och', 'om', 'oss', 'på', 'samma', 'sedan', 'sig', 'sin', 'sina', 'sitta', 'själv', 'skulle', 'som', 'så', 'sådan', 'sådana', 'sådant', 'till', 'under', 'upp', 'ut', 'utan', 'vad', 'var', 'vara', 'varför', 'varifrån', 'varit', 'varje', 'vars', 'vart', 'vem', 'vems', 'vi', 'vid', 'vilka', 'vilkas', 'vilken', 'vilket', 'vår', 'våra', 'vårt', 'än', 'är', 'åt', 'över']


In [51]:
def make_sentences_from_tokens(doc):
    all_sentences = []
    all_tok_sentences = []
    all_tok_stop_sentences = []
    all_tok_lemma_sentences = []
    all_tok_lemma_stop_sentences = []
    for sentence in doc.sentences:
        current_sentence = []
        current_tok_sentence = []
        current_tok_stop_sentence = []
        current_tok_lemma_sentence = []
        current_tok_lemma_stop_sentence = []
        for word in sentence.words:
            # only add if character is letter or number (removes , . ? ! etc.)
            w  = re.sub('[^\sa-zåäöA-ZÅÄÖ0-9-,]', '', word.text)
            if len(w) > 0:
                current_sentence.append(w)
                current_tok_sentence.append(w.lower())
                current_tok_lemma_sentence.append(word.lemma)
                if not word.text in stop_words:
                    current_tok_stop_sentence.append(w.lower())
                    current_tok_lemma_stop_sentence.append(word.lemma)
        
        sent = ' '.join(current_sentence)
        all_sentences.append(sent.lower())
        all_tok_sentences.append(current_tok_sentence)
        all_tok_stop_sentences.append(current_tok_stop_sentence)
        all_tok_lemma_sentences.append(current_tok_lemma_sentence)
        all_tok_lemma_stop_sentences.append(current_tok_lemma_stop_sentence)
    return all_sentences, all_tok_sentences, all_tok_stop_sentences, all_tok_lemma_sentences, all_tok_lemma_stop_sentences

In [52]:
def get_correct_answers(df):
    correct_answers = []
    correct_answers_parsed = []
    correct_answers_parsed_tok = []
    correct_answers_parsed_stop = []
    correct_answers_parsed_lemma = []
    correct_answers_parsed_lemma_stop = []
    correct_answers_loc = []
    answer_reformulations = []
    questions_parsed = []
    questions_parsed_tok = []
    questions_parsed_stop = []
    questions_parsed_lemma = []
    questions_parsed_lemma_stop = []
    # parse out the correct answer in the choices column
    for index, row in df.iterrows():
        answers = row['choices']
        df_row = pd.DataFrame(answers)
        # Collect the correct answer, add it to list. Save refomulation for those that use it
        answer_row = df_row.loc[df_row['type'] == 'Correct answer']
        answer_reformulation = False
        if answer_row.iloc[0]['extra']:
            answer_reformulation = answer_row.iloc[0]['extra']['comment']
        
        # parse the answer the same way as the context is parsed.
        correct_answer = answer_row.iloc[0]['text']
        doc = nlp(correct_answer)
        d_1, d_2, d_3, d_4, d_5 = make_sentences_from_tokens(doc)
        correct_answer_parsed = d_1[0]
        correct_answer_parsed_tok = d_2[0]
        correct_answer_parsed_stop = d_3[0]
        correct_answer_parsed_lemma = d_4[0]
        correct_answer_parsed_lemma_stop = d_5[0]

        # parse the question the same way as the context is parsed.
        question_raw = row['question']
        q_doc = nlp(question_raw)
        q_1, q_2, q_3, q_4, q_5 = make_sentences_from_tokens(q_doc)
        question_parsed = q_1[0]
        question_parsed_tok = q_2[0]
        question_parsed_stop = q_3[0]
        question_parsed_lemma = q_4[0]
        question_parsed_lemma_stop = q_5[0]
        
        correct_answer_loc = answer_row.iloc[0]['start']
        correct_answers.append(correct_answer)
        correct_answers_parsed.append(correct_answer_parsed)
        correct_answers_parsed_tok.append(correct_answer_parsed_tok)
        correct_answers_parsed_stop.append(correct_answer_parsed_stop)
        correct_answers_parsed_lemma.append(correct_answer_parsed_lemma)
        correct_answers_parsed_lemma_stop.append(correct_answer_parsed_lemma_stop)
        correct_answers_loc.append(correct_answer_loc)
        answer_reformulations.append(answer_reformulation)
        questions_parsed.append(question_parsed)
        questions_parsed_tok.append(question_parsed_tok)
        questions_parsed_stop.append(question_parsed_stop)
        questions_parsed_lemma.append(question_parsed_lemma)
        questions_parsed_lemma_stop.append(question_parsed_lemma_stop)
   
    return correct_answers, correct_answers_parsed, correct_answers_parsed_tok, correct_answers_parsed_stop, correct_answers_parsed_lemma, correct_answers_parsed_lemma_stop, correct_answers_loc, answer_reformulations, questions_parsed, questions_parsed_tok, questions_parsed_stop, questions_parsed_lemma, questions_parsed_lemma_stop


In [53]:
# add the correct answers as a column to the DataFrame
df_train['correct_answer'], df_train['correct_answer_parsed'], df_train['correct_answer_parsed_tok'], df_train['correct_answer_parsed_stop'], df_train['correct_answer_parsed_lemma'], df_train['correct_answer_parsed_lemma_stop'], df_train['correct_answer_loc'], df_train['answer_reformulation'], df_train['question_parsed'],  df_train['question_parsed_tok'], df_train['question_parsed_stop'], df_train['question_parsed_lemma'], df_train['question_parsed_lemma_stop'] = get_correct_answers(df_train)
df_train.head()

,choices,context,question,correct_answer,correct_answer_parsed,correct_answer_parsed_tok,correct_answer_parsed_stop,correct_answer_parsed_lemma,correct_answer_parsed_lemma_stop,correct_answer_loc,answer_reformulation,question_parsed,question_parsed_tok,question_parsed_stop,question_parsed_lemma,question_parsed_lemma_stop
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?,ett tillstånd för att resa in och vara i ett land,ett tillstånd för att resa in och vara i ett land,"[ett, tillstånd, för, att, resa, in, och, vara...","[tillstånd, resa, in, land]","[en, tillstånd, för, att, resa, in, och, vara,...","[tillstånd, resa, in, land]",175,False,vad är ett visum,"[vad, är, ett, visum]","[vad, visum]","[vad, vara, en, visum]","[vad, visum]"
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?,ditt pass,ditt pass,"[ditt, pass]",[pass],"[du, pass]",[pass],156,False,vad ska du ta med när du hämtar ditt beslut,"[vad, ska, du, ta, med, när, du, hämtar, ditt,...","[vad, ska, ta, hämtar, beslut]","[vad, skola, du, ta, med, när, du, hämta, du, ...","[vad, skola, ta, hämta, beslut]"
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...,ett år,ett år,"[ett, år]",[år],"[en, år]",[år],1619,False,hur länge gäller ett uppehållstillstånd för be...,"[hur, länge, gäller, ett, uppehållstillstånd, ...","[hur, länge, gäller, uppehållstillstånd, besök...","[hur, länge, gälla, en, uppehållstillstånd, fö...","[hur, länge, gälla, uppehållstillstånd, besök,..."
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],589,False,vad är ett uppehållstillståndskort,"[vad, är, ett, uppehållstillståndskort]","[vad, uppehållstillståndskort]","[vad, vara, en, uppehållstillståndskort]","[vad, uppehållstillståndskort]"
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],673,False,vad är ett uppehållstillståndskort,"[vad, är, ett, uppehållstillståndskort]","[vad, uppehållstillståndskort]","[vad, vara, en, uppehållstillståndskort]","[vad, uppehållstillståndskort]"


In [54]:
# check in which sentence the answer can be found

def collect_sentence_number_statistics(df):
    idx_of_ans = []
    sentences_with_ans = []
    idx_of_ans_text = []
    total_num_sents = []
    ans_loc_frac = []
    all_context_sentences = []
    all_context_tok_word_sentences = []
    all_context_tok_word_stop_sentences = []
    all_context_tok_lemma_sentences = []
    all_context_tok_lemma_stop_sentences = []
    for index, row in df.iterrows():
        # iterate over all characters in the paragraph and find in which sentence the location is
        tot_chars = 0
        answer = row['correct_answer_parsed']
        answer_loc = int(row['correct_answer_loc'])
        text = row['context']
        # split the text into each sentence
        doc = nlp(text)
        sentences, tok_word_sent, tok_word_sent_stop, tok_lemma_sent, tok_lemma_sent_stop = make_sentences_from_tokens(doc)
        all_context_sentences.append(sentences)
        all_context_tok_word_sentences.append(tok_word_sent)
        all_context_tok_word_stop_sentences.append(tok_word_sent_stop)
        all_context_tok_lemma_sentences.append(tok_lemma_sent)
        all_context_tok_lemma_stop_sentences.append(tok_lemma_sent_stop)

        # find in which sentences the answer is. How to know if it is the answer to the correct question??
        found_indexes = []
        loc_idx = None
        sentence_with_ans = None
        for index, sent in enumerate(sentences):
            num_chars = len(sent)+1 # TODO: check how to do this correctly with the current parsing!!
            tot_chars += num_chars
            if not loc_idx and tot_chars > answer_loc: # only collect if not already found
                loc_idx = index
                sentence_with_ans = sent
            if answer in sent:
                found_indexes.append(index)
        if not loc_idx:
            # if did not find sentence with answer in the text, sentence must be at the end (with sentence parsing characters are removed)
            loc_idx = index
            sentence_with_ans = sent

        
        # Match the indexes with the indexes found in text
        if not loc_idx in found_indexes:
            if len(found_indexes) == 1:
                # replace with where the index was found in the text
                loc_idx = found_indexes[0]
                sentence_with_ans = sentences[loc_idx]
            elif len(found_indexes) > 1:
                diff = np.abs(np.array(found_indexes) - loc_idx)
                min_diff = np.min(diff)
                min_diff_idx = diff.tolist().index(min_diff)
                # replace the index with the one found in text that is closest
                loc_idx = found_indexes[min_diff_idx]
                sentence_with_ans = sentences[loc_idx]
            else:
                print('ALERT - answer not found!')
                print('sentence by index: ', sentence_with_ans)
                print('answer: ', answer)



        # append the found indexes to the array for all paragraphs
        idx_of_ans_text.append(found_indexes)
        sentences_with_ans.append(sentence_with_ans) # append the sentence with the correct answer
        idx_of_ans.append(loc_idx) # append the location of the answer!
        total_num_sents.append(len(sentences))
        fracs = loc_idx/len(sentences)
        ans_loc_frac.append(fracs)

    return idx_of_ans, sentences_with_ans, idx_of_ans_text, total_num_sents, ans_loc_frac, all_context_sentences, all_context_tok_lemma_sentences, all_context_tok_lemma_stop_sentences, all_context_tok_word_stop_sentences, all_context_tok_word_sentences

        
df_train['answer_location'], df_train['sent_with_ans'], df_train['answer_locations_text'], df_train['paragraph_len'], df_train['loc_frac'], df_train['context_parsed'], df_train['context_parsed_tok_lemma'], df_train['context_parsed_tok_lemma_stop'], df_train['context_parsed_tok_stop'], df_train['context_parsed_tok']  = collect_sentence_number_statistics(df_train)
df_train.head()

did not find loc by index!
answer_loc:  2062
tot_chars:  2061
did not find loc by index!
answer_loc:  2171
tot_chars:  2154
did not find loc by index!
answer_loc:  1747
tot_chars:  1745
did not find loc by index!
answer_loc:  1873
tot_chars:  1855
did not find loc by index!
answer_loc:  2427
tot_chars:  2411
did not find loc by index!
answer_loc:  1638
tot_chars:  1616


,choices,context,question,correct_answer,correct_answer_parsed,correct_answer_parsed_tok,correct_answer_parsed_stop,correct_answer_parsed_lemma,correct_answer_parsed_lemma_stop,correct_answer_loc,...,answer_location,sent_with_ans,answer_locations_text,paragraph_len,loc_frac,context_parsed,context_parsed_tok_lemma,context_parsed_tok_lemma_stop,context_parsed_tok_stop,context_parsed_tok
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?,ett tillstånd för att resa in och vara i ett land,ett tillstånd för att resa in och vara i ett land,"[ett, tillstånd, för, att, resa, in, och, vara...","[tillstånd, resa, in, land]","[en, tillstånd, för, att, resa, in, och, vara,...","[tillstånd, resa, in, land]",175,...,2,visum är ett tillstånd för att resa in och var...,[2],21,0.095238,"[visum för affärs- och konferensbesök, om du s...","[[visa, för, affär, och, konferensbesök], [om,...","[[visa, affär, konferensbesök], [om, skola, be...","[[visum, affärs-, konferensbesök], [om, ska, b...","[[visum, för, affärs-, och, konferensbesök], [..."
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?,ditt pass,ditt pass,"[ditt, pass]",[pass],"[du, pass]",[pass],156,...,2,när du ska hämta ditt beslut ska du ta med dit...,"[2, 3]",19,0.105263,"[eget företag, efter beslut beslutet skickas t...","[[egen, företag], [efter, beslut, beslut, skic...","[[egen, företag], [efter, beslut, beslut, skic...","[[eget, företag], [efter, beslut, beslutet, sk...","[[eget, företag], [efter, beslut, beslutet, sk..."
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...,ett år,ett år,"[ett, år]",[år],"[en, år]",[år],1619,...,18,så länge gäller tillståndet ett uppehållstills...,[18],25,0.720000,"[utbytesdoktorander intervju, du kommer att få...","[[utbytesdoktorand, intervju], [du, komma, att...","[[utbytesdoktorand, intervju], [du, komma, få,...","[[utbytesdoktorander, intervju], [du, kommer, ...","[[utbytesdoktorander, intervju], [du, kommer, ..."
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],589,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[eget företag, när vi har tagit emot din ansök...","[[egen, företag], [när, vi, ha, ta, emot, du, ...","[[egen, företag], [när, ta, emot, ansökan], [o...","[[eget, företag], [när, tagit, emot, ansökan],...","[[eget, företag], [när, vi, har, tagit, emot, ..."
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],673,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[flytta tillbaka till sverige, så ansöker du d...","[[flytta, tillbaka, till, Sverige], [så, ansök...","[[flytta, tillbaka, Sverige], [så, ansöka, du,...","[[flytta, tillbaka, sverige], [så, ansöker, du...","[[flytta, tillbaka, till, sverige], [så, ansök..."


In [55]:
# Save dataframes to file
df_train.to_pickle("./data_frames/df_train.pkl")